In [37]:
import pandas as pd

import re
import pickle
from pathlib import Path

In [2]:
base_data_path = Path('./data/')
metadata_path = list(base_data_path.glob('*books.pkl'))[0]
data_path = list(base_data_path.glob('*chapters.pkl'))[0]

with open(metadata_path, 'rb') as f:
    metadata = pickle.load(f)

with open(data_path, 'rb') as f:
    data = pickle.load(f)

In [3]:
chapters = pd.DataFrame(columns=['chapter_id', 'book_id', 'chapter_title', 'chapter_summary', 'source'], data=data)

In [4]:
chapters.head()

,chapter_id,book_id,chapter_title,chapter_summary,source
0,295,41,Новая школа. Голодная жизнь в городе,1948 год. Рассказчик окончил сельскую четырёхл...,1
1,296,41,Игра на деньги. Рассказчика избивают,"В компанию, которая играла на деньги в «чи́ку»...",1
2,297,41,Учительница узнаёт об игре на деньги,Утром рассказчик явился в школу с разбитой физ...,1
3,298,41,Уроки французского. Таинственная посылка,Для рассказчика «начались... мучительные и нел...,1
4,299,41,Игра в «присте́нок». Учительница проигрывает р...,Уроки французского на этом не кончились. Посте...,1


In [11]:
books_path = list(Path('./data/books/').glob('*.txt'))
books_names = [str(p).split('/')[-1].replace('.txt', '') for p in books_path]
books_names = {k: v for k, v in zip(books_names, books_path)}

metadata_info = []

for md in metadata:
    book_id, book_name, url_or_book_summ, source_id, author = md
    book_path = None
    if book_name in books_names:
        book_path = books_names[book_name]
    else:
        if book_name == '«Что делать?»':
            book_path = books_names["«Что делать»"]
    
    if book_path is not None:
        metadata_info.append(
            {
                'book_id': book_id,
                'book_name': book_name,
                'book_summ': url_or_book_summ,
                'source_id': source_id,
                'author': author,
                'book_path': str(book_path)
            }
        )

In [12]:
filtered_metadata = []

for md in metadata_info:
    ln = chapters[chapters['book_id'] == md['book_id']].shape[0]
    if ln != 0:
        filtered_metadata.append(md)
len(filtered_metadata)

76

In [29]:
dataset = []


def read_book(path):
    with open(path, 'r', encoding='utf-8') as f:
        data = f.read()

    return data


for md_sample in filtered_metadata:
    book_id = md_sample['book_id']
    data_book = chapters[chapters['book_id'] == book_id].to_dict('records')
    book_path = md_sample['book_path']
    raw_text = read_book(book_path)
    if isinstance(md_sample['book_summ'], str) and len(md_sample['book_summ']) == 0:
        md_sample['book_summ'] = None
    dataset.append(
        {
            'metadata': md_sample,
            'chapters': data_book,
            'raw_text': f'\n\n{raw_text}'
        }
    )



In [33]:
import json

save_dir = Path('./data/')

with open(save_dir / 'raw_dataset.json', 'w', encoding='utf-8') as f:
    json.dump(dataset, f)

In [38]:
def map_chapters(sample):
    raw_text = sample['raw_text']
    chapters = sample['chapters']
    splitted_text = [raw_text,]
    res = re.finditer('\n\n(\d|\d\d)\n', raw_text)
    for r in res:
        start, end = r.span()
        split_sym = raw_text[start:end]

        split_txt = splitted_text[-1]
        split_txt = split_txt.split(split_sym)
        print(len(split_txt))
        assert len(split_txt) == 2
        splitted_text.extend(split_txt)
    splitted_text = splitted_text[1:]
    assert len(chapters) == len(splitted_text), (len(chapters), len(splitted_text))
    return processed_text


processed_text = map_chapters(dataset[0])

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2


AssertionError: (17, 36)

In [21]:
map_chapters(dataset[0])

no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no
no


In [ ]:
class DataSample:
    def __init__(self, book_id, book_name, book_summary, source_id, author, book_path, raw_chapters) -> None:
        self.book_id = book_id
        self.book_name = book_name
        self.book_summary = book_summary
        self.source_id = source_id
        self.author = author
        self.book_path = book_path
        self.book_path = book_path




In [35]:
dataset[0]

{'metadata': {'book_id': 151,
  'book_name': '«Человек на часах»',
  'book_summ': None,
  'source_id': 2,
  'author': 'Н.С. Лесков',
  'book_path': 'data/books/«Человек на часах».txt'},
 'chapters': [{'chapter_id': 1085,
   'book_id': 151,
   'chapter_title': 'Глава 9',
   'chapter_summary': 'Свиньин решил ехать прямо к обер-полицмейстеру Кокошину. Всё знали, что он «умеет сделать из мухи слона, но так же легко умеет сделать из слона муху». Этот офицер благоволил всем шалунам в армии и охотно прикрывал их проступки, ведь считал их забавными. Был уже пятый час утра.',
   'source': 2},
  {'chapter_id': 1086,
   'book_id': 151,
   'chapter_title': 'Глава 10',
   'chapter_summary': 'Свиньин приехал к генералу и всё ему изложил. Пришел к выводу, что Кокошину уже всё доложили, но это было не так. Пристав просто не видел смысла тревожить начальство столь нелепым случаем, к тому же история инвалидного офицера была весьма сомнительна. Кокошин послал за приставом, инвалидным офицером и спасенным